In [1]:
#dependencies and setup
import pandas as pd
import os
import numpy as np


In [2]:
# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(os.path.join("resources", "purchase_data.csv"))
df.columns = df.columns.str.replace(' ','_')

In [3]:

print (df)

     Purchase_ID             SN  Age  Gender  Item_ID  \
0              0        Lisim78   20    Male      108   
1              1    Lisovynya38   40    Male      143   
2              2     Ithergue48   24    Male       92   
3              3  Chamassasya86   24    Male      100   
4              4      Iskosia90   23    Male      131   
..           ...            ...  ...     ...      ...   
775          775     Aethedru70   21  Female       60   
776          776         Iral74   21    Male      164   
777          777     Yathecal72   20    Male       67   
778          778        Sisur91    7    Male      101   
779          779      Ennrian78   24    Male       50   

                                     Item_Name  Price  
0    Extraction, Quickblade Of Trembling Hands   3.53  
1                            Frenzied Scimitar   1.56  
2                                 Final Critic   4.88  
3                                  Blindscythe   3.27  
4                                  

In [4]:
#total number of players
num_players = df.SN
total_players = len(num_players.unique().tolist())
# Display total number of Players
pd.DataFrame({"Total Number of Players": [total_players]})


,Total Number of Players
0,576


In [5]:
#Purchasing Analysis
df.columns



Index(['Purchase_ID', 'SN', 'Age', 'Gender', 'Item_ID', 'Item_Name', 'Price'], dtype='object')

In [6]:
# Purchase Analysis
# number of unique items
unique_items = df.Item_ID
num_ui = len(unique_items.unique().tolist())
#average price, number of purchases, total revenue
price = df.Price
ave_pp = '$' + str(round(price.mean(),2))
num_pur = len(price.tolist())
tot_rev = '$' + str(round(price.sum(),2))
#print purchase analysis
pd.DataFrame({'Unique Items':[num_ui], 'Average Purchase Price':[ave_pp], 'Number of Purchases': [num_pur], 
              'Total Revenue':[tot_rev]})




,Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [7]:
#Gender Demographics
gender = df.Gender

In [8]:
# Percentage and Count of Male, Female, Other / Non-Disclosed Players
counts = gender.value_counts()
percent = gender.value_counts(normalize=True).mul(100).round(1).astype(str)+ '%'
pd.DataFrame({'Total Count':counts, 'Percentage of Players':percent})


,Total Count,Percentage of Players
Male,652,83.6%
Female,113,14.5%
Other / Non-Disclosed,15,1.9%


In [9]:
### Age Demographics

#The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  #Purchase Count
  #Average Purchase Price
  #Total Purchase Value
  #Average Purchase Total per Person by Age Group

In [10]:

bins = [0, 10, 15, 20, 25, 30, 35, 40, 41]
labels = ['<10', '10-14', '15-19', '20-24', '25-30', '31-35', '36-40', '40+']
df['binned'] = pd.cut(df['Age'], bins=bins, labels=labels)

print (df)

     Purchase_ID             SN  Age  Gender  Item_ID  \
0              0        Lisim78   20    Male      108   
1              1    Lisovynya38   40    Male      143   
2              2     Ithergue48   24    Male       92   
3              3  Chamassasya86   24    Male      100   
4              4      Iskosia90   23    Male      131   
..           ...            ...  ...     ...      ...   
775          775     Aethedru70   21  Female       60   
776          776         Iral74   21    Male      164   
777          777     Yathecal72   20    Male       67   
778          778        Sisur91    7    Male      101   
779          779      Ennrian78   24    Male       50   

                                     Item_Name  Price binned  
0    Extraction, Quickblade Of Trembling Hands   3.53  15-19  
1                            Frenzied Scimitar   1.56  36-40  
2                                 Final Critic   4.88  20-24  
3                                  Blindscythe   3.27  20-24  


In [11]:
df1 = df.groupby('binned')['Price'].agg(['count', 'mean', 'sum'])
print(df1)


        count      mean     sum
binned                         
<10        32  3.405000  108.96
10-14      54  2.900000  156.60
15-19     200  3.107800  621.56
20-24     325  3.020431  981.64
25-30      77  2.875584  221.42
31-35      52  2.994423  155.71
36-40      33  3.404545  112.35
40+         2  3.270000    6.54


In [12]:
#average purchase total per person in each age group
den = df.groupby('binned')['SN'].agg(['nunique']).astype(int)
num = df.groupby('binned')['Price'].agg(['sum'])
answer = pd.merge(den, num, left_index=True, right_index=True)

purch_person = (answer['sum']/answer['nunique']).to_frame('age_avg_pp')
print(purch_person)

        age_avg_pp
binned            
<10       4.540000
10-14     3.819512
15-19     4.143733
20-24     4.231207
25-30     3.752881
31-35     4.208378
36-40     4.321154
40+       3.270000


In [13]:

age_demo = pd.merge(df1, purch_person, left_index=True, right_index=True)
#df.age_demo.rename(inplace=True, columns={'Purchase Count', 'Average Purchase Price', 'Total Puchase Value', 
                #'Ave Total Purchase per Person'})
print(age_demo)



        count      mean     sum  age_avg_pp
binned                                     
<10        32  3.405000  108.96    4.540000
10-14      54  2.900000  156.60    3.819512
15-19     200  3.107800  621.56    4.143733
20-24     325  3.020431  981.64    4.231207
25-30      77  2.875584  221.42    3.752881
31-35      52  2.994423  155.71    4.208378
36-40      33  3.404545  112.35    4.321154
40+         2  3.270000    6.54    3.270000


In [14]:
#Top 5 Spenders
df.columns



Index(['Purchase_ID', 'SN', 'Age', 'Gender', 'Item_ID', 'Item_Name', 'Price',
       'binned'],
      dtype='object')

In [15]:
#create a dataframe of sum of the stats
spenders = df.groupby('SN').sum().sort_values(by=['Price'])
print (spenders)

             Purchase_ID  Age  Item_ID  Price
SN                                           
Alo38                 63   20      125   1.00
Aidai61              282   21      155   1.01
Chanirra79           586   23      155   1.01
Isurria36            528   22       19   1.02
Irilis75             582   20       19   1.02
...                  ...  ...      ...    ...
Iskadarya95          713   60      321  13.10
Iral74              2285   84      518  13.62
Chamjask73          1306   66      339  13.83
Idastidru52         1999   96      527  15.45
Lisosia93           1630  125      451  18.96

[576 rows x 4 columns]


In [17]:
spenders= spenders.nlargest(5, 'Price')
spenders = spenders.drop(['Age', 'Item_ID','Purchase_ID'], axis=1)
print (spenders)

             Price
SN                
Lisosia93    18.96
Idastidru52  15.45
Chamjask73   13.83
Iral74       13.62
Iskadarya95  13.10


In [23]:
#basic calculations on top spenders
# Purchase Count = number of unique purchases per person = how many repeats a name has in
#the original data frame
df.columns
df['Count']= df.groupby('SN')['SN'].transform('count')
count = pd.merge(df['Count'],df)
count1 = df[df.SN == 'Lisosia93']
count2 = df[df.SN == 'Idastidru52']
count3 = df[df.SN == 'Chamjask73']
count4 = df[df.SN == 'Iral74']
count5 = df[df.SN == 'Iskadarya95']
pieces = (count1, count2, count3, count4, count5)
df_count = pd.concat(pieces, ignore_index=True)
df_count = pd.DataFrame({"Lisosia93": [5], "Idastidru52": [4], "Chamjask73": [3], "Iral74": [4], "Iskadarya95": [3]},
            index = [1, 2, 3, 4, 5])
count6 = df_count.transpose()

count6 = count6.astype('float64')

count6d = count6.drop([2, 3, 4, 5], axis=1)

print(count6d)




               1
Lisosia93    5.0
Idastidru52  4.0
Chamjask73   3.0
Iral74       4.0
Iskadarya95  3.0


In [26]:
top_5 = pd.merge(count6d, spenders, left_index=True, right_index=True)
top_5 = top_5.rename(columns={1: 'Purchase_Count', 'Price': 'Total_Purchase_Value'})
print(top_5)

# Average Price
# Total Purchase Value

             Purchase_Count  Total_Purchase_Value
Lisosia93               5.0                 18.96
Idastidru52             4.0                 15.45
Chamjask73              3.0                 13.83
Iral74                  4.0                 13.62
Iskadarya95             3.0                 13.10


In [28]:
#Average Price = Purchase_Count/Total_Purchase_Value
Ave_Price = ['Total_Purchase_Value']/['Purchase_Count']


print(Ave_Price)

TypeError: unsupported operand type(s) for /: 'list' and 'list'

In [ ]:

#Create summary data frame to hold the results
#	Purchase Count	Average Purchase Price	Total Purchase Value
#SN			
#Lisosia93	5	$3.79	$18.96
#Idastidru52	4	$3.86	$15.45
#Chamjask73	3	$4.61	$13.83
#Iral74	4	$3.40	$13.62
#Iskadarya95	3	$4.37	$13.10


In [ ]:
#sort the total purchase value column in descending order